<a href="https://colab.research.google.com/github/11daniel/HFStockAnalysis/blob/main/HFStockAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('secretName')

In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import os
from dotenv import load_dotenv

from transformers import pipeline
import google.generativeai as genai
import os
from dotenv import load_dotenv
import time


# Get list of S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    table = pd.read_html(url)[0]
    return table['Symbol'].tolist()

tickers = get_sp500_tickers()
print(f"Loaded {len(tickers)} tickers.")

HTTPError: HTTP Error 403: Forbidden

In [ ]:
from transformers import pipeline

# Load FinBERT model (finance-specific sentiment model)
sentiment = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# Example financial headline
headline = "Netflix Poised to Double Ad Revenue This Year, Wedbush Says"

print(sentiment(headline))
# [{'label': 'positive', 'score': 0.995}]


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Download stock data
data = yf.download("NFLX", start="2025-01-01", end="2025-10-20", interval="1d")

# Calculate indicators
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()

# Generate signals safely (handles NaN)
data['Signal'] = np.where(
    data['SMA_20'] > data['SMA_50'], "BUY",
    np.where(data['SMA_20'] < data['SMA_50'], "SELL", "HOLD")
)

# Show last rows
print(data[['Close','SMA_20','SMA_50','Signal']].tail())

# Plot with buy/sell markers
plt.figure(figsize=(12,6))
plt.plot(data['Close'], label='Close Price', alpha=0.8)
plt.plot(data['SMA_20'], label='20-day SMA', alpha=0.7)
plt.plot(data['SMA_50'], label='50-day SMA', alpha=0.7)

# Mark buy/sell
buy = data[data['Signal'] == 'BUY']
sell = data[data['Signal'] == 'SELL']
plt.scatter(buy.index, buy['Close'], marker='^', color='green', label='BUY', alpha=1)
plt.scatter(sell.index, sell['Close'], marker='v', color='red', label='SELL', alpha=1)

plt.title("NFLX Buy/Sell Strategy (SMA Crossover)")
plt.legend()
plt.show()


In [ ]:
import numpy as np

def final_decision(sentiment_label, price_signal):
    if sentiment_label == "positive" and price_signal == "BUY":
        return "BUY"
    elif sentiment_label == "negative" and price_signal == "SELL":
        return "SELL"
    else:
        return "HOLD"

headline = "Netflix Poised to Double Ad Revenue This Year, Wedbush Says"
sentiment_result = sentiment(headline)[0]['label']   # positive/negative/neutral
latest_price_signal = data['Signal'].iloc[-1]        # from SMA crossover

print(final_decision(sentiment_result, latest_price_signal))
